In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
import numpy as np
from numpy import array

import seaborn as sns
import matplotlib.pyplot as plt
import datetime

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import scipy
from scipy import stats,interpolate
from scipy.stats import skew,kurtosis,describe,variation,shapiro,levene,f_oneway
from scipy.special import kl_div
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

import warnings 
warnings.filterwarnings(action='ignore')

#import eli5
#from eli5.sklearn import PermutationImportance

import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import GridSearchCV,cross_validate
from sklearn import model_selection
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, KFold 
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier,IsolationForest,StackingClassifier
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,recall_score,confusion_matrix,f1_score,fbeta_score,roc_auc_score,classification_report,make_scorer,balanced_accuracy_score
from mlxtend.classifier import StackingClassifier,StackingCVClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler,Binarizer

from imblearn.over_sampling import SMOTE,ADASYN,BorderlineSMOTE,SVMSMOTE
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import RandomUnderSampler,EditedNearestNeighbours,NearMiss,TomekLinks,CondensedNearestNeighbour,ClusterCentroids

import random
import tensorflow 
import tensorflow as tf

random.seed(1234)
np.random.seed(1234)
tf.random.set_seed(1234)

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
def describex(data):
        data = pd.DataFrame(data)
        stats = data.describe()
        skewness = data.skew()
        kurtosis = data.kurtosis()
        skewness_df = pd.DataFrame({'skewness':skewness}).T
        kurtosis_df = pd.DataFrame({'kurtosis':kurtosis}).T
        Appended_Stats = pd.concat([stats,skewness_df,kurtosis_df],axis=0)
        return Appended_Stats

In [3]:
def get_clf_eval(y_test,pred=None,pred_proba=None):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred_proba)
    f1 = f1_score(y_test,pred)
    f2 = fbeta_score(y_test,pred,beta=2)
    balanced_acc = balanced_accuracy_score(y_test,pred)
    print('Confusion Matrix')
    print(confusion)
    print('\n')
    print('정확도:',accuracy.round(3),'정밀도:', precision.round(3),'재현율:',recall.round(3),'AUC:',roc_auc.round(3),'F1:',f1.round(3),'F2:',f2.round(3),'Balanced_Accuracy:',balanced_acc.round(3))

In [4]:
def get_eval_by_threshold(y_test,pred_proba_c1,thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임계값',custom_threshold)
        get_clf_eval(y_test,custom_predict,pred_proba_c1)
        print('\n')

In [5]:
#한글폰트 적용
import matplotlib.font_manager as fm
path = 'C:/Windows/Fonts/gulim.ttc'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

In [6]:
#### one hot 적용해보기 

In [7]:
LoadData = pd.read_csv('train.csv').head()
LoadData

PRODUCT_ID  Y_Class  Y_Quality        TIMESTAMP     LINE PRODUCT_CODE  X_1  \
0  TRAIN_000        1   0.533433  2022-06-13 5:14  T050304         A_31  NaN   
1  TRAIN_001        2   0.541819  2022-06-13 5:22  T050307         A_31  NaN   
2  TRAIN_002        1   0.531267  2022-06-13 5:30  T050304         A_31  NaN   
3  TRAIN_003        2   0.537325  2022-06-13 5:39  T050307         A_31  NaN   
4  TRAIN_004        1   0.531590  2022-06-13 5:47  T050304         A_31  NaN   

   X_2  X_3  X_4  X_5  X_6  X_7  X_8  X_9  X_10  X_11  X_12  X_13  X_14  X_15  \
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_16  X_17  X_18  X_19  X_20  X_21  X_22  X_23  X_24  X_25  X_26  X_27  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_28  X_29  X_30  X_31  X_32  X_33  X_34  X_35  X_36  X_37  X_38  X_39  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_40  X_41  X_42  X_43  X_44  X_45  X_46  X_47  X_48  X_49  X_50  X_51  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_52  X_53  X_54  X_55  X_56  X_57  X_58  X_59  X_60  X_61  X_62  X_63  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_64  X_65  X_66  X_67  X_68  X_69  X_70  X_71  X_72  X_73  X_74  X_75  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_76  X_77  X_78  X_79  X_80  X_81  X_82  X_83  X_84  X_85  X_86  X_87  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

   X_88  X_89  X_90  X_91  X_92  X_93  X_94  X_95  X_96  X_97  X_98  X_99  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN

### 1.)  Classification Y_Class

#### Y-Class
- 0 : 적정 기준 미달 
- 1 : 적합
- 2 : 적정 기준 초과 

In [8]:
#### Train Data
data = pd.read_csv('train.csv')
data = data.drop(['Y_Quality'],axis=1) 

data_copy = data.copy()
#data_copy['Y_Class'] = data_copy['Y_Class'].replace({0:1, 2:1, 1:0})

#data_object = data.select_dtypes(include='object')
#data_inputs = data.select_dtypes(include=[int,float]).fillna(data.select_dtypes(include=[int,float]).mean()).dropna(axis=1)
#data_inputs = data.select_dtypes(include=[int,float]).fillna(0) 

data = data.fillna(0)

#data = pd.concat([data_object,data_inputs],axis=1)
ids = data.nunique().values>1 # 획일값 제거 
data = data.loc[:,ids]

# 적정기준 미달 & 적정기준 초과 통합 
data['Y_Class'] = data['Y_Class'].replace({0:1, 2:1, 1:0})
print(data.shape)
print('\n')
print('제품번호 구분\n',data['PRODUCT_CODE'].value_counts())
print('\n')
print('PRODUCT_CODE = T_31\n',data[data['PRODUCT_CODE']== 'T_31']['Y_Class'].value_counts())
print('\n')
print('PRODUCT_CODE = A_31\n',data[data['PRODUCT_CODE']== 'A_31']['Y_Class'].value_counts())
print('\n')
print('PRODUCT_CODE = 0_31\n',data[data['PRODUCT_CODE']== 'O_31']['Y_Class'].value_counts())

(598, 2599)


제품번호 구분
 T_31    343
A_31    249
O_31      6
Name: PRODUCT_CODE, dtype: int64


PRODUCT_CODE = T_31
 0    285
1     58
Name: Y_Class, dtype: int64


PRODUCT_CODE = A_31
 1    131
0    118
Name: Y_Class, dtype: int64


PRODUCT_CODE = 0_31
 0    4
1    2
Name: Y_Class, dtype: int64


In [9]:
#### Test Data
#test = pd.read_csv('test.csv')

#test = test[data.drop(['Y_Class'],axis=1).columns]

#print(test.shape)
#print('\n')
#print('제품번호 구분\n',test['PRODUCT_CODE'].value_counts())

#### 전처리 

In [10]:
def ANOVA_Feature_Selection(Fillna):
    
    Data = data_copy
    Target = 'Y_Class' # 예측변수 Y 

    Input_Data = Data.select_dtypes(include=[int,float]).drop([Target],axis=1) # 입력변수 X 
    Input_Data = Input_Data.loc[:,Input_Data.nunique().values>1] # 획일값 제거 
    
    Train_Input, Test_Input, Train_Target, Test_Target = train_test_split(Input_Data,Data[Target],test_size=0.1,random_state=2021)
    
    if Fillna == 0:
        
        
        Train_Input = Train_Input.select_dtypes(include=[int,float]).fillna(0).dropna(axis=1)
        Test_Input = Test_Input.select_dtypes(include=[int,float]).fillna(0).dropna(axis=1)
    
    if Fillna == 'Stat':
        
        Train_Input = Train_Input.select_dtypes(include=[int,float]).fillna(Train_Input.select_dtypes(include=[int,float]).mean()).dropna(axis=1)
        Test_Input = Test_Input.select_dtypes(include=[int,float]).fillna(Test_Input.select_dtypes(include=[int,float]).mean()).dropna(axis=1)
        Train_Input = Train_Input[Test_Input.columns] # 결측값 제거한 변수 통일 
        
        
    num_col = Train_Input.columns
    cat_col = ['Y_Class']     
    mult_idx = pd.MultiIndex.from_product([num_col, cat_col],names=['x', 'Y_Class']) 
    
    df = pd.concat([Train_Input,Train_Target],axis=1)

    anova_tables = []

    for x, grp in mult_idx:
        model = ols('{} ~ {}'.format(x, grp), data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=1)
        anova_tables.append(anova_table)

    df_anova_tables = pd.concat(anova_tables, keys=mult_idx, axis=0)

    significance_anova_table = df_anova_tables[df_anova_tables['PR(>F)'] < 0.05 ] # 0.05 유의수준 
    anova_feature_selection = significance_anova_table.reset_index()['x'].to_list()
    X_train = Train_Input[anova_feature_selection]
    X_test =  Test_Input[anova_feature_selection]
    y_train = Train_Target
    y_test = Test_Target

    return  X_train, y_train, X_test, y_test  
    
X_train, y_train, X_test, y_test   = ANOVA_Feature_Selection(Fillna='Stat')

In [11]:
X_train.shape,X_test.shape

((538, 1383), (60, 1383))

In [12]:
y_train.value_counts(),y_test.value_counts()

(1    366
 2     95
 0     77
 Name: Y_Class, dtype: int64,
 1    41
 0    11
 2     8
 Name: Y_Class, dtype: int64)

In [13]:
# 활용할 ML 나열 
rf = RandomForestClassifier(random_state=2021)
rf_bal = RandomForestClassifier(random_state=2021,class_weight='balanced')
ada = AdaBoostClassifier(random_state=2021)
gbm = GradientBoostingClassifier(random_state=2021,n_estimators=150)
xgb = XGBClassifier(random_state=2021)
xgb_bal = XGBClassifier(random_state=2021)
lgbm =  LGBMClassifier(random_state=2021)
lgbm_bal = LGBMClassifier(random_state=2021) 

models = [rf,rf_bal,ada,gbm,xgb,lgbm]

#### ML 

In [14]:
def test_model(model,Fillna):
    
    if Fillna == 0:
        
        X_train, y_train, X_test, y_test   = ANOVA_Feature_Selection(Fillna=0)
        
        X_train,y_train = SMOTEENN(random_state=2021).fit_resample(X_train,y_train)
        X_test,y_test = RandomUnderSampler(random_state=2021).fit_resample(X_test,y_test)
        
        
    if Fillna == 'Stat':
        
        X_train, y_train, X_test, y_test   = ANOVA_Feature_Selection(Fillna='Stat')
        
        X_train,y_train = SMOTEENN(random_state=2021).fit_resample(X_train,y_train)
        X_test,y_test = RandomUnderSampler(random_state=2021).fit_resample(X_test,y_test)

    
    ml = model
    ml.fit(X_train,y_train)      
    pred = ml.predict(X_test)
    pred_proba = ml.predict_proba(X_test)[:,1]
    print('----------------------------------------------------------------------------------------------------------')
    print(confusion_matrix(y_test,pred))
    print(classification_report(y_test, pred,target_names=['Fault1','Normal','Fault2']))
    
    if model == rf:
        print('RandomForest Prediction Result')
        print('\n')
        
    elif model == rf_bal:
        print('RandomForest(Balanced Weight) Prediction Result')
        print('\n')  
        
    elif model == ada:
        print('AdaBoost Prediction Result')
        print('\n')
     
    elif model == gbm:
        print('GradientBoostingMachine Prediction Result')
        print('\n')
        
    elif model == xgb:
        print('XGBoost Prediction Result')
        print('\n') 
        
    elif model == lgbm:
        print('LightGBM Prediction')
        print('\n')   

In [15]:
# 전체 모델  원큐 출력 
for i in range(0,len(models)):

    model = models[i]
    test_model(model,Fillna=0) 

----------------------------------------------------------------------------------------------------------
[[3 2 3]
 [0 7 1]
 [1 1 6]]
              precision    recall  f1-score   support

      Fault1       0.75      0.38      0.50         8
      Normal       0.70      0.88      0.78         8
      Fault2       0.60      0.75      0.67         8

    accuracy                           0.67        24
   macro avg       0.68      0.67      0.65        24
weighted avg       0.68      0.67      0.65        24

RandomForest Prediction Result


----------------------------------------------------------------------------------------------------------
[[3 2 3]
 [0 7 1]
 [1 1 6]]
              precision    recall  f1-score   support

      Fault1       0.75      0.38      0.50         8
      Normal       0.70      0.88      0.78         8
      Fault2       0.60      0.75      0.67         8

    accuracy                           0.67        24
   macro avg       0.68      0.67      0.65 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [16]:
# 전체 모델  원큐 출력 
for i in range(0,len(models)):

    model = models[i]
    test_model(model,Fillna='Stat') 

----------------------------------------------------------------------------------------------------------
[[4 0 4]
 [0 0 8]
 [0 0 8]]
              precision    recall  f1-score   support

      Fault1       1.00      0.50      0.67         8
      Normal       0.00      0.00      0.00         8
      Fault2       0.40      1.00      0.57         8

    accuracy                           0.50        24
   macro avg       0.47      0.50      0.41        24
weighted avg       0.47      0.50      0.41        24

RandomForest Prediction Result


----------------------------------------------------------------------------------------------------------
[[4 0 4]
 [0 0 8]
 [0 0 8]]
              precision    recall  f1-score   support

      Fault1       1.00      0.50      0.67         8
      Normal       0.00      0.00      0.00         8
      Fault2       0.40      1.00      0.57         8

    accuracy                           0.50        24
   macro avg       0.47      0.50      0.41 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [17]:
#X_train, y_train, X_test, y_test   = ANOVA_Feature_Selection(Fillna=0)
        
#X_train,y_train = SMOTEENN(random_state=2021).fit_resample(X_train,y_train)
#X_test,y_test = RandomUnderSampler(random_state=2021).fit_resample(X_test,y_test)

#def GBM_objective(trial):
#    params = {
#    'n_estimators':trial.suggest_int('n_estimators', 100, 200),
#    'subsample': trial.suggest_float('subsample', 0.8, 1),
#    'max_depth': trial.suggest_int('max_depth', 1,10),
#    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
#    'learning_rate': trial.suggest_float('learning_rate',0.001, 0.1),
#    'min_impurity_decrease': trial.suggest_float('min_impurity_decrease',0,2),
#    'max_leaf_nodes': trial.suggest_int('max_leaf_nodes',2,5)}
        
    
#    model = GradientBoostingClassifier (**params,random_state=2021)
#    skf = StratifiedKFold(n_splits=5)    
#    model.fit(X_train,y_train)    
#    score = cross_val_score(model,X_train,y_train, cv=skf, scoring="accuracy")
#    mean = score.mean()

#    return mean

#GBM_study = optuna.create_study(sampler = TPESampler(seed=2021),direction='maximize',study_name='GBM_Optuna')
#GBM_study.optimize(GBM_objective, n_trials=20)

#optimized_GBM = GradientBoostingClassifier(random_state=2021,**GBM_study.best_params)

#trial_gbm = GBM_study.best_trial
#trial_gbm_params = trial_gbm.params
#print('Best Trial: score {},\nparams {}'.format(trial_gbm.value, trial_gbm_params))



#optimized_GBM.fit(X_train,y_train)
#pred = optimized_GBM.predict(X_test)
#print(confusion_matrix(y_test,pred))
#print(classification_report(y_test, pred,target_names=['Fault1','Normal','Fault2']))    